In [1]:
import random
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import timm

class SRCNN(nn.Module):
    def __init__(self) -> None:
        super(SRCNN, self).__init__()
        # Feature extraction layer.
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, (9, 9), (1, 1), (4, 4)),
            nn.ReLU(True)
        )

        # Non-linear mapping layer.
        self.map = nn.Sequential(
            nn.Conv2d(64, 32, (5, 5), (1, 1), (2, 2)),
            nn.ReLU(True)
        )

        # Rebuild the layer.
        self.reconstruction = nn.Conv2d(32, 3, (5, 5), (1, 1), (2, 2))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self._forward_impl(x)

    # Support torch.script function.
    def _forward_impl(self, x: torch.Tensor) -> torch.Tensor:
        out = self.features(x)
        out = self.map(out)
        out = self.reconstruction(out)

        return out

srmodel = torch.load("best_psnr.pt", map_location = "cpu")
model = timm.create_model("convnext_base.fb_in22k", pretrained = True, num_classes = 25)

/home/tlrkrwlsmd/.local/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class combnet(nn.Module):
    def __init__(self, srmodel, model):
        super(combnet, self).__init__()
        self.srmodel = srmodel
        self.model = model
    
    def forward(self, x):
        # super resolution
        x = self.srmodel(x)
        x = self.model(x)
        return x
Model = combnet(srmodel, model)
Model

combnet(
  (srmodel): SRCNN(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
      (1): ReLU(inplace=True)
    )
    (map): Sequential(
      (0): Conv2d(64, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): ReLU(inplace=True)
    )
    (reconstruction): Conv2d(32, 3, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  )
  (model): ConvNeXt(
    (stem): Sequential(
      (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
    )
    (stages): Sequential(
      (0): ConvNeXtStage(
        (downsample): Identity()
        (blocks): Sequential(
          (0): ConvNeXtBlock(
            (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
            (norm): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=128, out_features=512, bias=True)
     

In [3]:
random_tensor = torch.rand([64, 3, 224, 224])
x = model(random_tensor)
x.size()
# x = x.view(x.size(0), -1)
# x.size()

torch.Size([64, 25])

In [4]:
CFG = {
    "LEARNING_RATE": 1e-4,
    "EPOCHS": 20,
    "BATCH_SIZE": 32,
    "DEVICE": torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
}

In [5]:
class ImageSet(Dataset):
    def __init__(self, img, transform = None, class_name = None, label = None):
        self.img = img
        self.label = label
        self.transform = transform
        self.class_name = class_name
        
    def __len__(self):
        return len(self.img)
    
    def __getitem__(self, idx):
        image = self.img[idx]
        label = self.label[idx]
        image = cv2.imread(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        label = class_name[label]
        return image, label

In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.Resize([224, 224])                    
])
data = pd.read_csv("train.csv")
trainset, valset, _, _ = train_test_split(data, data["label"], test_size = 0.2, stratify = data["label"], random_state = 0)
trainset = trainset.reset_index()
trainset.drop(["index"], axis = 1, inplace = True)
valset = valset.reset_index()
valset.drop(["index"], axis = 1, inplace = True)

In [7]:
classes = np.unique(data["label"])
class_name = {name: i for i, name in enumerate(classes)}

In [8]:
trainset = ImageSet(img = trainset["img_path"], transform = transform, class_name = class_name, label = trainset["label"])
validset = ImageSet(img = valset["img_path"], transform = transform, class_name = class_name, label = valset["label"])

In [9]:
trainloader = DataLoader(trainset, batch_size = CFG["BATCH_SIZE"], shuffle = True, num_workers = 0)
validloader = DataLoader(validset, batch_size = CFG["BATCH_SIZE"], shuffle = False, num_workers = 0)

In [10]:
for i, j in trainloader:
    print(i.size())
    print(j.size())
    print(model(i).size())
    break

torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 25])


In [11]:
optimizer = optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,\
    threshold_mode='abs', min_lr=1e-8, verbose=True)

def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            # validation score를 기준으로 scheduler를 조정한다
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
            print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}], Best Val F1 Score : [{_val_score:.5f}]')
    
    return best_model

In [12]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='macro')
    
    return _val_loss, _val_score

In [13]:
infer_model = train(model, optimizer, trainloader, validloader,\
    scheduler, device = CFG["DEVICE"])
torch.save(infer_model, "best_model_CONVNEXT_30epochs_large_srcnn.pt")

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 99/99 [00:27<00:00,  3.59it/s]


Epoch [1], Train Loss : [0.44768] Val Loss : [0.18108] Val F1 Score : [0.95142]
Epoch [1], Train Loss : [0.44768], Best Val F1 Score : [0.95142]


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 99/99 [00:26<00:00,  3.72it/s]


Epoch [2], Train Loss : [0.06877] Val Loss : [0.13919] Val F1 Score : [0.95990]
Epoch [2], Train Loss : [0.06877], Best Val F1 Score : [0.95990]


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 99/99 [00:29<00:00,  3.32it/s]


...

Epoch [18], Train Loss : [0.00012] Val Loss : [0.17055] Val F1 Score : [0.96141]


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 99/99 [00:27<00:00,  3.57it/s]


Epoch [19], Train Loss : [0.00011] Val Loss : [0.17080] Val F1 Score : [0.96141]


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 99/99 [00:27<00:00,  3.66it/s]


Epoch [20], Train Loss : [0.00011] Val Loss : [0.17111] Val F1 Score : [0.96173]
Epoch 00020: reducing learning rate of group 0 to 3.1250e-06.


In [14]:
class TestSet(Dataset):
    def __init__(self, img, transform = None):
        self.img = img
        self.transform = transform
        
    def __len__(self):
        return len(self.img)
    
    def __getitem__(self, idx):
        image = self.img[idx]
        image = cv2.imread(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        return image
transform_ = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.Resize([224, 224])                    
])
test = pd.read_csv("test.csv")
test_set = TestSet(img = test["img_path"], transform = transform_)
test_loader = DataLoader(test_set, batch_size = 1, shuffle = False)

In [15]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            pred = model(imgs)
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    return preds


In [16]:
preds = inference(model, test_loader, device = CFG["DEVICE"])
classes = list(class_name.keys())
final = []
for pred in preds:
    final.append(classes[pred])
submit = pd.read_csv("./sample_submission.csv")
submit["label"] = final
submit.to_csv("./submit_30epochs_large_srcnn.csv", index = False)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 6786/6786 [03:14<00:00, 34.92it/s]
